In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

artists_data_path = "../data/artists.jsonl"
sessions_data_path = "../data/sessions.jsonl"
track_storage_data_path = "../data/track_storage.jsonl"
tracks_data_path = "../data/tracks.jsonl"
users_data_path = "../data/users.jsonl"

artists_data_frame = pd.read_json(artists_data_path, lines=True)
sessions_data_frame = pd.read_json(sessions_data_path, lines=True)
tracks_data_frame = pd.read_json(tracks_data_path, lines=True)
users_data_frame = pd.read_json(users_data_path, lines=True)

# Mergowanie podobnych gatunków muzycznych (artists)
TODO


# Scalenie danych

In [ ]:
merged_data_frame = pd.merge(sessions_data_frame, tracks_data_frame, left_on="track_id",
                             right_on="id")
merged_data_frame = pd.merge(merged_data_frame, users_data_frame, left_on="user_id",
                             right_on="user_id")
merged_data_frame = pd.merge(merged_data_frame, artists_data_frame, left_on="id_artist",
                             right_on="id")
merged_data_frame.head(5)


## Wyliczenie, czy dana piosenka w danej sesji została pominięta

In [ ]:
# delete rows with event_type other than PLAY and SKIP
merged_data_frame = merged_data_frame[merged_data_frame["event_type"].isin(["PLAY", "SKIP"])]

# TODO moze to trzeba liczyc inaczej?

# if for one user_id and track_id and session_id there is only PLAY event, then the track was not skipped
merged_data_frame["skipped"] = merged_data_frame.groupby(["user_id", "track_id", "session_id"])[
  "event_type"].transform(lambda x: x != "PLAY").astype(bool)

# if there are 2 rows with the same user_id, track_id and session_id, and different "skipped" value, remove the row with "skipped" = 0
merged_data_frame = merged_data_frame.drop_duplicates(subset=["user_id", "track_id", "session_id"],
                                                      keep="last")

## Usunięcie niepotrzebnych kolumn

In [ ]:
merged_data_frame = merged_data_frame.drop(
  columns=["id_x", "id_y", "premium_user", "name_x", "name_y", "city", "street", "id_artist",
           "event_type", "timestamp", "user_id", "track_id", "session_id"])
merged_data_frame.head(10)

In [ ]:
merged_data_frame.info()

In [ ]:
merged_data_frame.describe()

In [ ]:
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import metrics
from scipy.stats import entropy

df = merged_data_frame.copy()


print("Entropy of skipped column: ", str(round(entropy(df["skipped"].value_counts()), 4)))

print("MI(skipped, genres): ", str(round(metrics.mutual_info_score(df["skipped"], df["encoded_genres"]), 4)))

df.head(5)

In [ ]:
# save to jsonl file
merged_data_frame.to_json("../data/merged_data.jsonl", orient="records", lines=True)
